# Feature Selection using Particle Swarm Optimization (PSO)

The search space for this problem corresponds to the power set of the set of the 54 attributes, which means
$2^{54}$ possibilities. Denote such space by $\mathcal{S}$. The Particle Swarm Optimization (PSO) algorithm can seek the desired subset of attributes without having
to pass through each of the elements in this colossus space. The strategy is to allows solutions (particles) in fixed-size population to move in the solutions space according to the best solution found by itself and to the best solution found
so far by any of the individuals.

Before submitting this problem to the PSO algorithm, it is necessary to define the way solutions (subsets) will
be represented and the function to rank the solutions, also known as the *fitness function*.

## Solutions representation

For this problem, a solution $i$ will be represented by a vector $x^i \in [0,1]^{54}$, in which each component $j$, $x^i_j$, means the probability of selecting the corresponding attribute $j$. Denote $[0,1]^{54}$ by $\mathcal{S_2}$. A threshold $0 \leq \theta \leq 1$ defines 
if the attribute was selected or not: $x^i_j \geq \theta$ means that the attribute $j$ is selected in the solution $i$,
as implemented by the simple function below. In this implementation, $\theta = 0.5$:

In [1]:
def is_selected(xij):
    '''
    Determines if attribute j in solution i was selected
    based on the value on position j of vector x^i.
    '''
    return xij >= 0.5

## Fitness function

This function will rank each solution according to the objective of selecting the most appropriate attribute for this classification problem. Since the purpose is to determine the class of an instance based on the attribute values, the similarity measure of correlation will be applied here. Correlation between two random variables $X$ and $Y$ is given by the
equation:

$$
corr(X,Y) = \frac{cov(X,Y)}{\sigma_X\sigma_Y}
$$

where $cov(X,Y)$ is the covariance between $X$ and $Y$, and $\sigma_X, \sigma_Y$ are the stardard deviations for $X$ and $Y$ respectively. The fitness function is defined as the mean of the unsigned correlations (in $[0,1]$) of each selected attribute with respect to the class. Given that the PSO algorithm used here seeks to minimize the fitness value, the fitness function $f:\mathcal{S_2}\to\mathbb{R}$ is given by the equation:

$$
f(x^i) = 1 - \left(\frac{\sum_{j, x^i_j = 1} |corr(x^i_j,cover\_type)|}{\sum_{j, x^i_j=1} 1}\right).
$$

## Implementation

First of all, it is necessary to load the dataset:

In [2]:
import pandas as pd
# read the dataset
dataset = pd.read_csv("datasets/covertype_norm_train.csv")
# preview
dataset.head()

,elevation,aspect,slope,horiz_dist_hydro,vert_dist_hydro,horiz_dist_road,hillshade_9,hill_shade_noon,hill_shade_15,horiz_dist_fire,...,soil_type_31,soil_type_32,soil_type_33,soil_type_34,soil_type_35,soil_type_36,soil_type_37,soil_type_38,soil_type_39,cover_type
0,-0.573753,-0.518424,-0.428658,0.436024,-0.475092,-0.979056,0.927864,0.144520,-0.534162,-0.220768,...,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,3
1,1.656009,-0.010549,0.868502,-0.516497,-0.280544,1.817610,0.862413,0.665801,-0.534162,2.273548,...,-0.214265,4.938531,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,7
2,0.169501,-0.799569,0.632655,0.455170,1.891910,-0.388051,0.796962,-1.245563,-1.335438,-0.687429,...,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,5
3,-1.205043,1.268208,1.576043,0.234990,1.648725,-0.649457,-2.933743,-0.159560,1.956291,-0.501856,...,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,6
4,-1.057345,0.152697,0.986425,0.134472,0.530073,-1.041945,0.404256,1.056762,-0.014415,-0.794770,...,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,3


Correlations with respect to the class are fixed values, and can be calculated using the following (note that possible NaN are removed):

In [3]:
# compute attribute-class correlations
class_correlations = dataset.corr(method="pearson")['cover_type']
# fill possible NaN values with 0
class_correlations.fillna(0, inplace=True)

Now, define the fitness function by:

In [4]:
import numpy as np

def f(x, theta=0.5):
    '''
    Takes a vector in [0,1]^54 and compute its fitness value.
    '''
    selected_attrs = list(map(is_selected, x))
    if (any(selected_attrs)):
        sum_corr = sum([abs(class_correlations[i]) \
                        for i in np.arange(0,dataset.shape[1]-1) if selected_attrs[i]])
        count_attrs = sum(selected_attrs)
        return 1 - (sum_corr/count_attrs)
    else:
        return 1

The PSO implementation used here comes from the `pyswarm` library. An amount of 30 executions is performed for each parameter combinations:

In [5]:
from pyswarm import pso
import itertools
# define variables's lower bound
lb = np.zeros(dataset.shape[1] - 1)
# define variables' upper bound
ub = np.ones(dataset.shape[1] - 1)
# number of trials
max_trials = 30
# swarm size
swarm_sizes = [25, 50, 100]
# maximum iterations
max_iterations = [50, 100, 200]
# dataframe to store results
results_columns = pd.Index(['swarm_size', 
                            'max_iterations',
                            'fitness']).append(dataset.columns[:-1])
results = pd.DataFrame(columns=results_columns)
# running PSO for max_trials times combining parameters
for swarm_size, max_iterations in itertools.product(*[swarm_sizes, max_iterations]):
    print("swarm_size = ", 
          str(swarm_size),
          ", max_iterations = ",
          str(max_iterations))
    # execute PSO for the selected parameters
    for trial in range(1, max_trials + 1):
        # execute PSO
        xopt, fopt = pso(f, lb, ub, swarmsize=swarm_size, maxiter=max_iterations)
        # booleanize vector
        xopt = list(map(is_selected, xopt))
        # print info
        print("Trial ", str(trial),
              ": ", str(fopt),
              ", selected ", str(sum(xopt)))
        # append result
        results = results.append(pd.Series([swarm_size, 
                                            max_iterations, 
                                            fopt] + xopt,
                                           index=results_columns), 
                                 ignore_index=True)

swarm_size =  25 , max_iterations =  50
Stopping search: maximum iterations reached --> 50
Trial  1 :  0.8730081348853301 , selected  15
Stopping search: maximum iterations reached --> 50
Trial  2 :  0.8651603956523894 , selected  16
Stopping search: maximum iterations reached --> 50
Trial  3 :  0.8962256652219494 , selected  24
Stopping search: maximum iterations reached --> 50
Trial  4 :  0.8819893398697056 , selected  20
Stopping search: maximum iterations reached --> 50
Trial  5 :  0.8842887087035751 , selected  18
Stopping search: maximum iterations reached --> 50
Trial  6 :  0.8820794507362089 , selected  23
Stopping search: maximum iterations reached --> 50
Trial  7 :  0.8922073040547845 , selected  24
Stopping search: maximum iterations reached --> 50
Trial  8 :  0.8861698995223343 , selected  20
Stopping search: maximum iterations reached --> 50
Trial  9 :  0.8993125413007471 , selected  26
Stopping search: maximum iterations reached --> 50
Trial  10 :  0.8816668474421719 , se

Stopping search: maximum iterations reached --> 200
Trial  24 :  0.8813065269375533 , selected  16
Stopping search: maximum iterations reached --> 200
Trial  25 :  0.880307039360832 , selected  18
Stopping search: maximum iterations reached --> 200
Trial  26 :  0.8855049961255301 , selected  21
Stopping search: maximum iterations reached --> 200
Trial  27 :  0.8816572904358294 , selected  20
Stopping search: maximum iterations reached --> 200
Trial  28 :  0.887792942541403 , selected  19
Stopping search: maximum iterations reached --> 200
Trial  29 :  0.8925286803602643 , selected  21
Stopping search: maximum iterations reached --> 200
Trial  30 :  0.8959730508861755 , selected  20
swarm_size =  50 , max_iterations =  50
Stopping search: maximum iterations reached --> 50
Trial  1 :  0.8799386709296831 , selected  22
Stopping search: maximum iterations reached --> 50
Trial  2 :  0.8842463515946045 , selected  22
Stopping search: maximum iterations reached --> 50
Trial  3 :  0.8756130149

Stopping search: maximum iterations reached --> 200
Trial  17 :  0.8783061396336741 , selected  19
Stopping search: maximum iterations reached --> 200
Trial  18 :  0.8804632327899482 , selected  20
Stopping search: maximum iterations reached --> 200
Trial  19 :  0.8942790715609448 , selected  20
Stopping search: maximum iterations reached --> 200
Trial  20 :  0.8833296031801348 , selected  19
Stopping search: maximum iterations reached --> 200
Trial  21 :  0.8833852194692279 , selected  18
Stopping search: maximum iterations reached --> 200
Trial  22 :  0.8879191856655902 , selected  22
Stopping search: maximum iterations reached --> 200
Trial  23 :  0.8748268920525184 , selected  16
Stopping search: maximum iterations reached --> 200
Trial  24 :  0.8698583686915726 , selected  17
Stopping search: maximum iterations reached --> 200
Trial  25 :  0.8651842801345939 , selected  15
Stopping search: maximum iterations reached --> 200
Trial  26 :  0.8658113924554499 , selected  12
Stopping s

Stopping search: maximum iterations reached --> 200
Trial  10 :  0.8512161686145514 , selected  16
Stopping search: maximum iterations reached --> 200
Trial  11 :  0.8673544408118247 , selected  15
Stopping search: maximum iterations reached --> 200
Trial  12 :  0.8568005289357922 , selected  15
Stopping search: maximum iterations reached --> 200
Trial  13 :  0.8636431283061025 , selected  14
Stopping search: maximum iterations reached --> 200
Trial  14 :  0.8770789166489991 , selected  17
Stopping search: maximum iterations reached --> 200
Trial  15 :  0.8421358926250204 , selected  11
Stopping search: maximum iterations reached --> 200
Trial  16 :  0.8626906690952313 , selected  16
Stopping search: maximum iterations reached --> 200
Trial  17 :  0.8530394324354885 , selected  15
Stopping search: maximum iterations reached --> 200
Trial  18 :  0.8407042206227588 , selected  13
Stopping search: maximum iterations reached --> 200
Trial  19 :  0.8567409743651313 , selected  14
Stopping s

Now, check the solution with the least fitness values:

In [6]:
# take a solution with minimum fitness
selected = results.loc[results['fitness'] == results['fitness'].min()]
selected

,swarm_size,max_iterations,fitness,elevation,aspect,slope,horiz_dist_hydro,vert_dist_hydro,horiz_dist_road,hillshade_9,...,soil_type_30,soil_type_31,soil_type_32,soil_type_33,soil_type_34,soil_type_35,soil_type_36,soil_type_37,soil_type_38,soil_type_39
216,100,100,0.815857,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False


Finally, store the results in a CSV for further usage:

In [7]:
# write the results in a csv
results.to_csv('results/pso_selected_attributes.csv', index=False)